In [2]:
#MODULOS DE SELENIUM
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime

#MODULOS SECUNDARIOS
import pandas as pd
import numpy as np
import os
import time
import ssl
import sys
import traceback

#MODULO ALERTAS DESACTIVADAS
import warnings
warnings.filterwarnings('ignore')

#MODULO PROPIOS
from utils import modelamiento

In [2]:
#CAPTURA DE LA LISTA DE POSTS
URL_POST  = pd.read_excel('url/URL_POST.xlsx') 
lista_url = list(URL_POST['url_facebook'])

In [3]:
#FUNCION POST / VIDEO FACEBOOK
def ManagerFaceBookScrapper (url,medio):

    driver = webdriver.Firefox()
    driver.get(url)


    
    try:
        time.sleep(1)
        # Espera a que el botón sea clickeable
        boton = driver.find_element(By.XPATH, value="/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/div").click()
        time.sleep(8)

        comentarios = driver.find_elements(By.CSS_SELECTOR,"div.xv55zj0.x1vvkbs.x1rg5ohu.xxymvpz") #CLASE DEL CONTENEDOR DE NOMBRE DE USUARIO, COMENTARIO, HORA Y REACCIONES


    # div.xv55zj0.x1vvkbs.x1rg5ohu.xxymvpz CLASE EN LOS VIDEOS


    except:
        try:
            time.sleep(1)
            # Espera a que el botón sea clickeable
            boton = driver.find_element(By.XPATH, "//div[@role='button' and contains(., 'Más pertinentes')]")
            boton.click()

            time.sleep(2)
            boton = driver.find_element(By.XPATH, "//span[contains(text(), 'Todos los comentarios')]")
            boton.click()
        except Exception as e:
            pass
            # print("no_button")
            # print("Ocurrió un error:", e)

        time.sleep(10)

        # Encontrar todos los elementos con la clase específica
        classCSScomentarios_ = "div.x1r8uery.x1iyjqo2.x6ikm8r.x10wlt62.x1pi30zi"

        comentarios = driver.find_elements(By.CSS_SELECTOR,classCSScomentarios_) #CLASE DEL CONTENEDOR DE NOMBRE DE USUARIO, COMENTARIO, HORA Y REACCIONES


    # Extraer los textos de los comentarios

    textos_comentarios = [comentario.text for comentario in comentarios]

    try:
        procesados         = [modelamiento(texto) for texto in textos_comentarios]
        flattened_data = [item for sublist in procesados for item in sublist]

        # Crear un DataFrame dividiendo cada elemento en "Usuario" y "Comentario"

        df = pd.DataFrame(flattened_data, columns=["Usuario", "Comentario", "Hora", "Reacciones"])
        df["url_facebook"] = url
        # df = pd.DataFrame([comentario.split('\n', 1) for comentario in textos_comentarios], columns=['Usuario', 'Comentario'])

        # Guardar en un archivo Excel

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'data/data_{medio}_{timestamp}.xlsx'
        # filename = f'data_{medio}_{timestamp}.xlsx'
        df.to_excel(filename, index=False)
        
    except:
        pass

    # print(textos_comentarios)
    driver.quit()
    print("Excel generado con éxito 🎉")    

In [ ]:
#EJECUCION DE LA FUNCION A TODAS LAS URLS CAPTURADAS PARA EL DIA

for url in lista_url:
    ManagerFaceBookScrapper(url,medio="medios")

Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
Excel generado con éxito 🎉
E

In [3]:
#LIMPIEZA DATAFRAME Y CONSOLIDADO DIARIO INTERPRETALAB
carpeta = 'data'

archivos = [f for f in os.listdir(carpeta) if f.endswith((".xlsx",".xls"))]
dataframes = []

for archivo in archivos:
    ruta_completa = os.path.join(carpeta,archivo)
    df = pd.read_excel(ruta_completa)
    dataframes.append(df)

df_final = pd.concat(dataframes,ignore_index=True)

mask = df_final["Reacciones"] != 0  # Filtrar filas donde "Número" no sea 0

for idx in df_final[mask].index:  # Iterar sobre las filas filtradas
    partes = df_final.at[idx, "Comentario"].rsplit("\n", 2)  # Dividir de derecha a izquierda
    if len(partes) > 1:
        df_final.at[idx, "Comentario"] = partes[0]  # Asignar el comentario
        df_final.at[idx, "Hora"] = partes[1]        # Asignar la hora
    else:
        df_final.at[idx, "Hora"] = "Pendiente"  # Si no hay salto de línea, dejar "Pendiente"


timestamp = datetime.now().strftime("%d%m%Y")
filename = f'data consolidado/facebook_{timestamp}.xlsx'

# df_final['Codigo'] = df_final['url_facebook'].str.split('/').str[3].astype('float') # crea la columna codigo/que es el ID del Medio de comunicación

In [ ]:
# #Codigos de Medios
# codigos = pd.read_excel(r'documentacion\ecuador.xlsx',sheet_name='facebook')[['Codigo','Nombre']]
# codigos['Codigo'] = codigos['Codigo'].astype('float')

In [4]:
# data_consolidado = pd.merge(

#     left     = df_final,
#     right    = codigos,
#     left_on  = 'Codigo',
#     right_on = 'Codigo',
    # how      = 'left'

# ).drop(columns='Codigo')

df_final.to_excel(filename,index=False)